In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import gmaps
from api_keys import g_key
from ipywidgets.embed import embed_minimal_html

In [9]:
montreal_collisions = pd.read_csv('Resources/montreal_2012_2018.csv')
montreal_incomes = pd.read_csv('Resources/montreal_incomes.csv')
montreal_districts = pd.read_csv('Resources/montreal_districts.csv')

In [3]:
montreal_collisions=montreal_collisions.rename(columns={'LOC_LONG':'Longitude',
                                                        'LOC_LAT':'Latitude',
                                                        'AN': 'Year',
                                                        'DT_ACCDN': 'Date',
                                                        'HR_ACCDN':'Hour',
                                                        'JR_SEMN_ACCDN':'Weekday',
                                                        'CD_COND_METEO':'Weather Conditions',
                                                        'CD_MUNCP':'District'
                                                       })

In [4]:
montreal_collisions= montreal_collisions.loc[montreal_collisions["GRAVITE"]== "Mortel"]
montreal_collisions= montreal_collisions.loc[montreal_collisions["REG_ADM"]== "Montréal(06)"]

In [5]:
montreal_collisions=pd.merge(montreal_collisions, montreal_districts, on= "District")

In [6]:
montreal_collisions['Hour'] = montreal_collisions['Hour'].str[:2]
montreal_collisions['Hour']=montreal_collisions['Hour'].replace('No',99)
montreal_collisions['Hour']=montreal_collisions['Hour'].astype(int)

In [7]:
montreal_data=montreal_collisions[['Longitude', 'Latitude', 'Year','Date','Hour','Districts']]
montreal_data=montreal_data.sort_values('Year',ascending=False)
montreal_data.reset_index(drop=True)

,Longitude,Latitude,Year,Date,Hour,Districts
0,-73.677218,45.518355,2018,2018/09/10,21,Montréal
1,-73.550944,45.547617,2018,2018/07/08,22,Montréal
2,-73.600069,45.544772,2018,2018/06/19,6,Montréal
3,-73.645704,45.529995,2018,2018/05/11,2,Montréal
4,-73.629950,45.541305,2018,2018/11/04,10,Montréal
...,...,...,...,...,...,...
174,-73.613588,45.539950,2012,2012/05/29,14,Montréal
175,-73.605014,45.539364,2012,2012/01/16,18,Montréal
176,-73.616715,45.551361,2012,2012/11/27,12,Montréal
177,-73.829372,45.467596,2012,2012/02/02,16,Pointe-Claire


In [14]:
final_montreal_data=pd.merge(montreal_data,montreal_incomes, how='outer',on='Districts')
final_montreal_data= final_montreal_data.rename(columns={'Median total income of households 2015':'Median Income'})
final_montreal_data=final_montreal_data.dropna()
final_montreal_data

,Longitude,Latitude,Year,Date,Hour,Districts,Median Income,Population
0,-73.677218,45.518355,2018.0,2018/09/10,21.0,Montréal,"50,227","1,704,694"
1,-73.550944,45.547617,2018.0,2018/07/08,22.0,Montréal,"50,227","1,704,694"
2,-73.600069,45.544772,2018.0,2018/06/19,6.0,Montréal,"50,227","1,704,694"
3,-73.645704,45.529995,2018.0,2018/05/11,2.0,Montréal,"50,227","1,704,694"
4,-73.629950,45.541305,2018.0,2018/11/04,10.0,Montréal,"50,227","1,704,694"
...,...,...,...,...,...,...,...,...
174,-73.793593,45.484392,2015.0,2015/07/30,10.0,Dollard-Des Ormeaux,"78,981","48,899"
175,-73.788746,45.503413,2012.0,2012/02/21,20.0,Dollard-Des Ormeaux,"78,981","48,899"
176,-73.632316,45.513745,2013.0,2013/12/08,11.0,Mont-Royal,"109,540","20,276"
177,-73.850314,45.447270,2013.0,2013/10/28,6.0,Kirkland,"115,381","20,151"


In [42]:
montreal_hoods=pd.DataFrame(final_montreal_data.groupby('Districts').count()).reset_index()
hoods=montreal_hoods[["Districts","Hour"]]
hoods= hoods.rename(columns={"Hour": "Number of Collisions"})
final_montreal=pd.merge(hoods, montreal_incomes, how='outer',on='Districts')
final_montreal["City"]= "Montreal"
final_montreal=final_montreal.rename(columns={'Districts': Neighbourhoods})

NameError: name 'Neighbourhoods' is not defined

In [43]:
final_montreal

,Districts,Number of Collisions,Median total income of households 2015,Population,City
0,Côte-Saint-Luc,2.0,"58,935","32,448",Montreal
1,Dollard-Des Ormeaux,2.0,"78,981","48,899",Montreal
2,Dorval,3.0,"64,689","18,980",Montreal
3,Hampstead,2.0,"122,496","6,973",Montreal
4,Kirkland,2.0,"115,381","20,151",Montreal
5,Mont-Royal,1.0,"109,540","20,276",Montreal
6,Montréal,155.0,"50,227","1,704,694",Montreal
7,Montréal-Est,2.0,"50,318","3,850",Montreal
8,Montréal-Ouest,2.0,"115,029","5,050",Montreal
9,Pointe-Claire,4.0,"80,242","31,380",Montreal


In [40]:
final_montreal.to_csv('Output/final_montreal.csv')

In [ ]:
income_analysis= final_montreal_data.groupby("Hour").count()
income_analysis=income_analysis.drop(99.0, axis=0)
income_analysis=income_analysis.reset_index()
income_analysis

In [ ]:
final_montreal_data.groupby('Median Income').count()

In [ ]:
# Histogram for data collected for median income
plt.scatter(income_analysis['Hour'],income_analysis['Year'])
plt.title('Fatal Collision Data')
plt.savefig('Output/districts_hist.png')
#p_value=sts.normaltest(final_montreal_data['Median Income']) 
#print(f' p-value is {p_value[1]}; Normally distributed')

In [37]:
gmaps.configure(api_key=g_key)

In [38]:
locations = final_montreal_data[["Latitude", "Longitude"]].astype(float)
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))